In [5]:
import os
import cv2
import numpy as np

In [6]:
open_pic_path = next(os.walk("img/OpenFace"))[2]
open_pic_path = [x for x in open_pic_path if ".jpg" in x]
len(open_pic_path)

1231

In [7]:
close_pic_path = next(os.walk("img/ClosedFace"))[2]
close_pic_path = [x for x in close_pic_path if ".jpg" in x]
len(close_pic_path)

1192

In [8]:
train_open_pic_path = open_pic_path[:1181]
train_close_pic_path = close_pic_path[:1142]
test_open_pic_path = open_pic_path[1181:]
test_close_pic_path = close_pic_path[1142:]

In [9]:
train_picture_path = train_open_pic_path+ train_close_pic_path
test_picture_path = test_open_pic_path + test_close_pic_path

In [10]:
from tensorflow.keras.models import load_model
import torchvision.transforms.functional as TF
import torch

In [11]:
def preprocess_image(image):
    image = TF.to_pil_image(image)
    image = TF.resize(image, (128, 128))
    image = TF.to_tensor(image)
    image = (image - image.min())/(image.max() - image.min())
    image = (2 * image) - 1
    image = torch.transpose(image,0,2).detach().numpy()
    return image

## Open = 1
## Closed = 0

In [12]:
def generate_combine_data(pp_):
    X = []
    Y = []
    for path in pp_:
        if "closed" in path:
            img = cv2.imread(f"img/ClosedFace/{path}")
            Y.append(0)
        else:
            img = cv2.imread(f"img/OpenFace/{path}")    
            Y.append(1)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        preprocessed_image = preprocess_image(img)
        X.append(preprocessed_image)
    return np.array(X),np.array(Y)

In [13]:
train_inputs,train_labels = generate_combine_data(train_picture_path)
test_inputs,test_labels = generate_combine_data(test_picture_path)

In [19]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [16]:
model = Xception(weights=None,input_shape=(128,128,1), classifier_activation="softmax",classes=2)

In [17]:
model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   288         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [20]:
model.compile(
              optimizer = 'adam',
              loss = SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy'],
             )

In [21]:
model.fit(train_inputs, train_labels, epochs=25,  validation_split = 0.1)

Epoch 1/25
66/66 [==============================] - 16s 135ms/step - loss: 0.7351 - accuracy: 0.6742 - val_loss: 0.6931 - val_accuracy: 0.4335
Epoch 2/25
66/66 [==============================] - 7s 109ms/step - loss: 0.2802 - accuracy: 0.8942 - val_loss: 0.6758 - val_accuracy: 1.0000
Epoch 3/25
66/66 [==============================] - 7s 105ms/step - loss: 0.1277 - accuracy: 0.9490 - val_loss: 0.6456 - val_accuracy: 1.0000
Epoch 4/25
66/66 [==============================] - 7s 108ms/step - loss: 0.0714 - accuracy: 0.9740 - val_loss: 0.6387 - val_accuracy: 1.0000
Epoch 5/25
66/66 [==============================] - 7s 109ms/step - loss: 0.0715 - accuracy: 0.9755 - val_loss: 0.5221 - val_accuracy: 1.0000
Epoch 6/25
66/66 [==============================] - 7s 109ms/step - loss: 0.0321 - accuracy: 0.9900 - val_loss: 0.4467 - val_accuracy: 1.0000
Epoch 7/25
66/66 [==============================] - 7s 108ms/step - loss: 0.0485 - accuracy: 0.9798 - val_loss: 0.3116 - val_accuracy: 1.0000
Epoch

In [27]:
def get_confusion(inputs,labels):
    TP = 0
    FN = 0
    TN = 0
    FP = 0
    for inp,label in zip(inputs,labels):
        predict = np.array(model.predict(np.expand_dims(inp,0))[0])
        predict = np.argmax(predict)
        if label ==1 and predict == label:
            TP += 1
        if label ==1 and predict != label:
            FN += 1
        if label ==0 and predict == label:
            TN += 1
        if label ==0 and predict != label:
            FP += 1
    ACC = (TP+TN) / (TP+FP+FN+TN)
    PRECISION = TP / (TP+FP)
    RECALL = TP / (TP+FN)
    F1_SCORE = 2/((1/PRECISION) + (1/RECALL))
    return ACC,PRECISION,RECALL,F1_SCORE

In [23]:
ACC,PRECISION,RECALL,F1_SCORE = get_confusion(train_inputs,train_labels)

In [24]:
print(f"Train ACC --- {ACC}")
print(f"Train PRECISION --- {PRECISION}")
print(f"Train RECALL --- {RECALL}")
print(f"Train F1_SCORE --- {F1_SCORE}")

Train ACC --- 0.9969866551872578
Train PRECISION --- 0.9941077441077442
Train RECALL --- 1.0
Train F1_SCORE --- 0.9970451667370199


In [25]:
ACC,PRECISION,RECALL,F1_SCORE = get_confusion(test_inputs,test_labels)

In [26]:
print(f"Test ACC --- {ACC}")
print(f"Test PRECISION --- {PRECISION}")
print(f"Test RECALL --- {RECALL}")
print(f"Test F1_SCORE --- {F1_SCORE}")

Test ACC --- 0.98
Test PRECISION --- 1.0
Test RECALL --- 0.96
Test F1_SCORE --- 0.9795918367346937
